# Flang [AArch64](https://en.wikipedia.org/wiki/AArch64)

My personal notes on Flang AArch64

- [Flang](https://flang.llvm.org/docs/) release [15.0.07, 2023-01-12](https://github.com/llvm/llvm-project/releases/tag/llvmorg-15.0.7)
- JupyterLab 3.5.2
- [Termux 0.118.0](https://termux.dev/en/)
- Moto G4 with [Android 9.0_r44 ARM64 AOSiP](https://forum.xda-developers.com/t/rom-9-0_r44-arm64-android-open-source-illusion-project-athene-unofficial.3889942/)
- Processor: Snapdragon 617 octa-core ARMv8 Cortex-A53 64-bit
    - Two quad-core clusters 1.2/1.5 GHz

2023-01-28

In [1]:
!inxi

CPU: 2x 4-core AArch64 (-MCP AMP-) speed/min/max: 435/499:403/1651:1210 MHz
Kernel: 3.10.108-lk.r17_rev aarch64 Up: 3d 19h 25m Mem: 1178.7/1843.6 MiB
(63.9%) Storage: 14.56 GiB (197.3% used) Procs: 7 Shell: python3.11
inxi: 3.3.24


In [2]:
!inxi -C

CPU:
  Info: 2x 4-core model: AArch64 bits: 64 type: MCP AMP
  Speed (MHz): avg: 435 min/max: 499:403/1651:1210 cores: 1: 499 2: 499
    3: 403 4: 403 5: 403 6: 403


In [4]:
!nproc --all

8


In [14]:
!lscpu

Architecture:            aarch64
  CPU op-mode(s):        32-bit, 64-bit
  Byte Order:            Little Endian
CPU(s):                  8
  On-line CPU(s) list:   0,2,4-7
  Off-line CPU(s) list:  1,3
Vendor ID:               ARM
  Model name:            Cortex-A53
    Model:               4
    Thread(s) per core:  1
    Core(s) per cluster: 3
    Socket(s):           -
    Cluster(s):          2
    Stepping:            r0p4
    CPU(s) scaling MHz:  56%
    CPU max MHz:         1651.2000
    CPU min MHz:         0.0000
    Flags:               fp asimd evtstrm aes pmull sha1 sha2 crc32


In [16]:
!cat /proc/cpuinfo

Processor	: AArch64 Processor rev 4 (aarch64)
processor	: 0
processor	: 1
processor	: 2
processor	: 3
processor	: 4
processor	: 5
processor	: 6
processor	: 7
Features	: fp asimd evtstrm aes pmull sha1 sha2 crc32 
CPU implementer	: 0x41
CPU architecture: 8
CPU variant	: 0x0
CPU part	: 0xd03
CPU revision	: 4

Hardware	: Qualcomm Technologies, Inc MSM8952
Revision	: 82ad
Serial		: a740b00000000000
Device		: athene_13mp
Radio		: Brazil
MSM Hardware	: MSM8952


Feature | Description
- | -
fp 	    | floating point (VFPv4) present
asimd 	| advanced SIMD (Neon) present
evtstrm | timer event stream generation
aes 	| AESD and AESE instructions
pmull 	| PMULL, PMULL2 instructions
sha1 	| SHA1* instructions
sha2 	| SHA256* instructions
crc32 	| CRC32* instructions

- ARM Cortex-A53 implements the ARMv8-A 64-bit instruction set
- 2-wide decode superscalar processor, capable of dual-issuing some instructions
- VFPv4 has 32 64-bit FPU registers and fused multiply-accumulate instructions, per core
- In Neon, the SIMD 
    - supports up to 16 operations at the same time
    - shares the same floating-point registers as used in VFP
    - support 128-bit vectors, but will execute with 64 bits at a time
    - gcc needs -funsafe-math-optimizations to work with

In [30]:
!pkg show flang

Package: flang
Version: 15.0.7
Maintainer: @termux
Installed-Size: 535 MB
Depends: libc++, libllvm, clang, lld, mlir
Homepage: https://flang.llvm.org/
Download-Size: 27.0 MB
APT-Manual-Installed: yes
APT-Sources: https://mirrors.hit.edu.cn/termux/apt/termux-main stable/main aarch64 Packages
Description: LLVM's Fortran frontend



In [29]:
!flang-new --version

flang-new version 15.0.7
Target: aarch64-unknown-linux-android24
Thread model: posix
InstalledDir: /data/data/com.termux/files/usr/bin


\
from https://fortran-lang.org/en/learn/quickstart/hello_world/ :

In [17]:
%%writefile hello.f90
program main
  ! This is a comment line; it is ignored by the compiler
  print *, 'Hello, World!'
end

Overwriting hello.f90


In [18]:
!flang-new hello.f90 -o hello

In [11]:
!./hello

 Hello, World!


In [12]:
!ls -lh hello

-rwx------ 1 u0_a113 u0_a113 1.7M Jan 27 23:23 hello


In [25]:
%%bash
flang-new hello.f90 -o hello \
-O3 -s -fno-asynchronous-unwind-tables  \
-ffunction-sections -fdata-sections -Wl,--gc-sections,--strip-all

In [28]:
!strip --strip-all --verbose -R .comment hello

copy from `hello' [elf64-littleaarch64] to `stmbIon9' [elf64-littleaarch64]


In [29]:
!./hello

 Hello, World!


In [30]:
!ls -lh hello

-rwx------ 1 u0_a113 u0_a113 209K Jan 27 23:26 hello


In [33]:
!objdump -h hello


hello:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn
  0 .interp       00000015  00000000000002a8  00000000000002a8  000002a8  2**0
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  1 .note.android.ident 00000098  00000000000002c0  00000000000002c0  000002c0  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  2 .dynsym       00000420  0000000000000358  0000000000000358  00000358  2**3
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  3 .gnu.version  00000058  0000000000000778  0000000000000778  00000778  2**1
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  4 .gnu.version_r 00000040  00000000000007d0  00000000000007d0  000007d0  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  5 .gnu.hash     0000001c  0000000000000810  0000000000000810  00000810  2**3
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  6 .dynstr       000001d1  000000000000082c  000

In [65]:
%%writefile array.f90
program main
  implicit none
  integer :: i, j, k, N=3
  real :: a(3,3,3)=reshape([ .50, .73, .22, .29, .65, .41, .69, .25,  &
    .76, .64, .60, .73, .93, .24, .63, .19, .73, .77, .93, .70, .29,  &
    .53, .34, .20, .91, .02, .47 ], shape(a), order=[3,2,1])
  write(*,"(*(xg0.2))") (((a(k,j,:)),new_line('a'),j=1,N),new_line('a'),k=1,N)
  write(*,"(*(xg0.2))") ((a(k,:,:)),new_line('a'),k=1,N)
end

Overwriting array.f90


In [68]:
!flang-new array.f90 && ./a.out

 .50 .73 .22 
 .29 .65 .41 
 .69 .25 .76 
 
 .64 .60 .73 
 .93 .24 .63 
 .19 .73 .77 
 
 .93 .70 .29 
 .53 .34 .20 
 .91 .20E-1 .47 
 

 .50 .29 .69 .73 .65 .25 .22 .41 .76 
 .64 .93 .19 .60 .24 .73 .73 .63 .77 
 .93 .53 .91 .70 .34 .20E-1 .29 .20 .47 



In [71]:
%%writefile exit.f90
program main
    call exit(42)
end

Overwriting exit.f90


In [78]:
!flang-new exit.f90

In [83]:
!strip --strip-all --verbose a.out

copy from `a.out' [elf64-littleaarch64] to `stxB91wh' [elf64-littleaarch64]


In [84]:
! ./a.out ; echo $?

42


In [85]:
!wc -c a.out

407592 a.out


In [86]:
!stat -c "%s %n" a.out

407592 a.out


In [87]:
!size a.out

   text	   data	    bss	    dec	    hex	filename
 391950	  12848	     88	 404886	  62d96	a.out


In [88]:
!objdump -x a.out


a.out:     file format elf64-littleaarch64
a.out
architecture: aarch64, flags 0x00000150:
HAS_SYMS, DYNAMIC, D_PAGED
start address 0x00000000000213b4

Program Header:
    PHDR off    0x0000000000000040 vaddr 0x0000000000000040 paddr 0x0000000000000040 align 2**3
         filesz 0x0000000000000268 memsz 0x0000000000000268 flags r--
  INTERP off    0x00000000000002a8 vaddr 0x00000000000002a8 paddr 0x00000000000002a8 align 2**0
         filesz 0x0000000000000015 memsz 0x0000000000000015 flags r--
    LOAD off    0x0000000000000000 vaddr 0x0000000000000000 paddr 0x0000000000000000 align 2**12
         filesz 0x00000000000203b4 memsz 0x00000000000203b4 flags r--
    LOAD off    0x00000000000203b4 vaddr 0x00000000000213b4 paddr 0x00000000000213b4 align 2**12
         filesz 0x000000000003fa1c memsz 0x000000000003fa1c flags r-x
    LOAD off    0x000000000005fdd0 vaddr 0x0000000000061dd0 paddr 0x0000000000061dd0 align 2**12
         filesz 0x0000000000003018 memsz 0x0000000000003018 flags rw-

In [89]:
!objdump -d a.out


a.out:     file format elf64-littleaarch64


Disassembly of section .text:

00000000000213b4 <.text>:
   213b4:	d503249f 	bti	j
   213b8:	d280001d 	mov	x29, #0x0                   	// #0
   213bc:	d280001e 	mov	x30, #0x0                   	// #0
   213c0:	910003e0 	mov	x0, sp
   213c4:	14000001 	b	213c8 <__libc_init@plt-0x3f6e8>
   213c8:	d503233f 	paciasp
   213cc:	d100c3ff 	sub	sp, sp, #0x30
   213d0:	a9027bfd 	stp	x29, x30, [sp, #32]
   213d4:	910083fd 	add	x29, sp, #0x20
   213d8:	f0000208 	adrp	x8, 64000 <strcmp@plt+0x3240>
   213dc:	f0000209 	adrp	x9, 64000 <strcmp@plt+0x3240>
   213e0:	f000020a 	adrp	x10, 64000 <strcmp@plt+0x3240>
   213e4:	f0000202 	adrp	x2, 64000 <strcmp@plt+0x3240>
   213e8:	910023e3 	add	x3, sp, #0x8
   213ec:	aa1f03e1 	mov	x1, xzr
   213f0:	f945e508 	ldr	x8, [x8, #3016]
   213f4:	f945e929 	ldr	x9, [x9, #3024]
   213f8:	f945ed4a 	ldr	x10, [x10, #3032]
   213fc:	a900a7e8 	stp	x8, x9, [sp, #8]
   21400:	f9000fea 	str	x10, [sp, #24]
   21404:	f945f042 	ldr	x2,